In [1]:
import os
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "2"

In [2]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import matplotlib.pyplot as plt
import seaborn as sns
import csv

import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler
from threading import Timer
from timeit import default_timer as timer

In [3]:
prep_dataset = pd.read_csv('../datasets/com_concept_drift/sdn_train_unormalized.csv', delimiter=",")


# Normalizing

In [4]:
def normalizing(df):
    f_columns = ['temperature','label']
    scaler1 = StandardScaler().fit(df)
    scaler2 = StandardScaler().fit(df)

    scaler1= scaler1.fit(df[f_columns].to_numpy())
    scaler2 = scaler2.fit(df[['delay']])

    df.loc[:,f_columns] = scaler1.transform(df[f_columns].to_numpy())
    df['delay'] = scaler2.transform(df[['delay']])
    return df





In [5]:
def LSTMconf(X_train):
    print('Init config LSTM')
    model = keras.Sequential()
    model.add(
        keras.layers.Bidirectional(
            keras.layers.LSTM(
                 activation="relu",
                units=40,
                input_shape=(X_train.shape)
            )
        ))
    model.add(keras.layers.Dense(units=40, activation="relu"))
    model.add(keras.layers.Dense(units=40, activation="relu"))
    model.add(keras.layers.Dense(units=40, activation="relu"))
    model.add(keras.layers.Dense(units=40, activation="relu"))
    model.add(keras.layers.Dropout(rate=0.2))
    model.add(keras.layers.Dense(units=1))
    
    loss ="mse"
    optim = tf.keras.optimizers.Adam(
    learning_rate=0.0001)
    metrics=["accuracy"]

    model.compile(loss=loss, optimizer=optim, 
             metrics=metrics
             )
    return model
    

In [9]:
def LSTMfit(model,X_train,Y_train):
    print('Init Train')
    start = timer()
    history = model.fit(
        X_train, Y_train, 
        epochs=100, 
        batch_size= 10,
        validation_split=0,
        shuffle=False,
    #     callbacks=[tensorboard_callback]
    )
    print('trraining duration: ',round(timer() - start))

In [12]:
def saveFile(dataset, name='dataset'):
    print('saving: ',name, '......')
    f = open(name,'w')
    try:
        writer = csv.writer(f)
        writer.writerow(dataset.columns)
        for i in np.arange(int(dataset.shape[0])):
            writer.writerow(dataset.iloc[i,])
    finally:
        f.close()
        
        
def modeling3D(X, y):
    print(X.iloc[3])
    Xs, ys = [], []
    v = [X.iloc[0:3].to_numpy()]
    Xs.append(v)
    ys.append([y])
    return np.array(Xs), np.array(ys)
        
def checkFileExistance(filePath):
    try:
        with open(filePath, 'r') as f:
            return True
    except FileNotFoundError as e:
        return False
    except IOError as e:
        return False
    
def getCurrencyDf(MyLIst):
    deltaT = np.max(MyLIst.temperature) - np.min(MyLIst.temperature)
    deltaTime = np.max(MyLIst.seconds) - np.min(MyLIst.seconds)
    meanDelay = np.mean(MyLIst.delay)
    label=MyLIst.iloc[0,3]
    cols = ['timestamp','temperature','label' ,'delay' ]
    if checkFileExistance('datasets/sdn_train_mininet_unormalized.csv'):
        cdf = pd.read_csv('datasets/sdn_train_mininet_unormalized.csv', delimiter=",")
    else:        
        cdf = pd.DataFrame(columns = cols)
    new_row = pd.DataFrame([[deltaTime,deltaT,label,meanDelay]], columns = cols)
    cdf = pd.concat([cdf, new_row], ignore_index=True)
    return cdf
def preprocessing(MyLIst):
    currency_df = getCurrencyDf(MyLIst)
    saveFile(currency_df, 'datasets/sdn_train_mininet_unormalized.csv')
    norm = normalizing(currency_df)
    saveFile(norm, 'datasets/sdn_train_mininet_normalized.csv')
    window_train = norm.iloc[norm.shape[0]-1,:]
    return modeling3D(window_train, window_train.delay)
    

In [13]:
window = 20
cont = 0;
while(cont<prep_dataset.shape[0]):
    x_train,y_train =preprocessing(prep_dataset.iloc[cont:cont+window,:])
model = LSTMconf(x_train)
LSTMfit(model,x_train, y_train)

# r = Timer(1.0, preprocessing, (prep_dataset.iloc[cont:cont+window,:]))
# r.start()
# print(X_train)
    

saving:  datasets/sdn_train_mininet_unormalized.csv ......
saving:  datasets/sdn_train_mininet_normalized.csv ......
0.0
Init config LSTM
Init Train
Epoch 1/100
1/1 [==============================] - 5s 5s/step - loss: 64276408.0000 - accuracy: 1.0000
Epoch 2/100
1/1 [==============================] - 0s 4ms/step - loss: 183735584.0000 - accuracy: 1.0000
Epoch 3/100
1/1 [==============================] - 0s 3ms/step - loss: 941109.2500 - accuracy: 1.0000
Epoch 4/100
1/1 [==============================] - 0s 5ms/step - loss: 220063264.0000 - accuracy: 1.0000
Epoch 5/100
1/1 [==============================] - 0s 6ms/step - loss: 29132316.0000 - accuracy: 1.0000
Epoch 6/100
1/1 [==============================] - 0s 6ms/step - loss: 160174032.0000 - accuracy: 1.0000
Epoch 7/100
1/1 [==============================] - 0s 5ms/step - loss: 146930928.0000 - accuracy: 1.0000
Epoch 8/100
1/1 [==============================] - 0s 4ms/step - loss: 3052492.7500 - accuracy: 0.0000e+00
Epoch 9/100
1/1

1/1 [==============================] - 0s 6ms/step - loss: 2116685.7500 - accuracy: 0.0000e+00
Epoch 77/100
1/1 [==============================] - 0s 7ms/step - loss: 21297576.0000 - accuracy: 0.0000e+00
Epoch 78/100
1/1 [==============================] - 0s 6ms/step - loss: 525741.9375 - accuracy: 1.0000
Epoch 79/100
1/1 [==============================] - 0s 6ms/step - loss: 11411790.0000 - accuracy: 1.0000
Epoch 80/100
1/1 [==============================] - 0s 7ms/step - loss: 6234934.5000 - accuracy: 0.0000e+00
Epoch 81/100
1/1 [==============================] - 0s 6ms/step - loss: 23681448.0000 - accuracy: 1.0000
Epoch 82/100
1/1 [==============================] - 0s 8ms/step - loss: 21923.4668 - accuracy: 0.0000e+00
Epoch 83/100
1/1 [==============================] - 0s 10ms/step - loss: 43731892.0000 - accuracy: 0.0000e+00
Epoch 84/100
1/1 [==============================] - 0s 7ms/step - loss: 45715332.0000 - accuracy: 1.0000
Epoch 85/100
1/1 [==============================] - 0s